In [1]:
import os
import cv2
import pandas as pd
import time
from PIL import Image
import numpy as np
import mediapipe as mp 
mpPose = mp.solutions.pose
pose = mpPose.Pose()
mpDraw = mp.solutions.drawing_utils # For drawing keypoints
points = mpPose.PoseLandmark # points will have all 33 landmarks like nose, eyes, ears etc.


In [2]:
mp_pose=mp.solutions.pose


In [3]:
data = list()
data.append('label')
rootdir = "/home/shikha/POSEESTIMATION/TRAIN"

"""for p in points: #p will iterate through nose, eyes, ears etc...
        x = str(p)[13:]
        data.append(x + "_x")
        data.append(x + "_y")
        data.append(x + "_z")
        data.append(x + "_vis")
df = pd.DataFrame(columns = data)"""
num_images=0



In [4]:
c=0
cols=['label','leftElbow','rightElbow','leftShoulder','rightShoulder','leftKnee','rightKnee']

In [5]:
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    #ON: c[1] is y coordinate of c and b[1] is the y coordinate of b. that way we calculate the angle in radians
    #ON: then convert radians to angle and then do 360-angle if it is greater than 180 since hand or anything wont move 360 degree.
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
    #if angle<0:
        #angle+=360
        
    return angle 

In [6]:
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        try:
            temp=[]    
            one=[]
            imagename=os.path.basename(subdir)
            img = cv2.imread(rootdir + "/" +imagename+"/" +file)
            imageWidth, imageHeight = img.shape[:2]
            imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            results = pose.process(imgRGB)
            if results.pose_landmarks:
                    mpDraw.draw_landmarks(img, results.pose_landmarks, mpPose.POSE_CONNECTIONS) #draw landmarks on image
                    landmarks = results.pose_landmarks.landmark
            one.append(imagename)
            leftElbow=calculate_angle([landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y],
                  [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y],
                   [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y])
            print(leftElbow)
            one.append(leftElbow)
            rightElbow=calculate_angle([landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y],
                       [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y],
                       [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y])
            one.append(rightElbow)


            leftShoulder=calculate_angle([landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y],
                       [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y],
                       [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y])
            one.append(leftShoulder)
            rightShoulder=calculate_angle([landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y],
                       [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y],
                       [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y])

            one.append(rightShoulder)
            leftKnee=calculate_angle([landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y],
                       [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y],
                       [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y])
            one.append(leftKnee)
            rightKnee=calculate_angle([landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y],
                       [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y],
                       [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y])
            one.append(rightKnee)
            if c==0:
                df=pd.DataFrame([list(one)],columns=cols)
                c=c+1
            else:
                df.loc[c]=one
                c=c+1
            num_images += 1
        
        except ValueError as ve:
            continue 
df.to_csv("train_converted.csv", index=False)

105.85073111147973
105.13414828844094
179.51989883317532
145.0215308055094
15.159118697190484
51.95206478967646
6.468142616638129
22.61751468557988
137.19902647179032
137.19902647179032
107.87200393913805
173.89801757100074
38.81252435262986
38.81252435262986
56.03320063142708
48.53909042037264
76.02176245310817
30.535661167622273
121.80669565607619
36.6799070118084
0.11089618804814466
178.20840759201596
67.14533504485789
67.14533504485789
176.21974905174898
174.4257316175658
126.49840145242788
118.2311880164572
77.45549463937063
103.55901931533907
109.00731901211495
78.61088312337247
104.51275605118175
134.34861582936355
52.49444733118714
44.30675928194905
35.62268282731338
175.76529845016483
108.4028223311929
179.63159423289358
117.33985208084226
28.7971366626635
79.01929761080416
79.01929761080416
119.83359747990525
91.66589032613706
79.53264674522035
159.51314267381557
159.51314267381557
106.79786947974063
111.36131456595817
54.72616622969406
178.95393706324447
93.83666683490617
83

160.4847702021152
152.97905681432223
159.7778276948591
159.7778276948591
167.95055080609322
171.53549890482935
167.18207293555807
70.01333710171626
86.55979553625171
170.64400206848242
147.52297247450767
147.52297247450767
167.569363411157
153.64581080774386
134.28852332779505
153.4094656423152
147.1409490304543
165.80267002241624
159.56839385836108
32.693032049909434
166.79849866285153
166.79849866285153
162.56859535826763
165.0998485969105
156.00265112605078
156.00265112605078
171.6231370625114
85.12652435655372
171.50793635134255
126.06713204149793
4.909890070120822
164.8777733792933
171.05568565491757
166.87462863089667
34.23783009555027
168.82723388938066
0.32336382407681535
178.13136923179982
178.13136923179982
161.0540212660373
149.72739796236797
168.20068255558843
176.38642414155697
174.3271692586212
147.70455736372588
26.428856794636886
163.29833046921422
155.9742689481525
162.191729296538
175.7793464048223
169.32972805117933
169.32972805117933
166.55041023689282
179.869840050

26.971827727965536
26.971827727965536
34.31998718118542
173.89250328024278
173.89250328024278
174.20393819249927
35.439840137212606
51.974099299727776
51.974099299727776
115.04808112955448
90.68562864322104
168.20321296270438
150.6776735818259
42.621446173491485
166.65310157364993
160.70507683608372
160.70507683608372
173.48842651944838
21.64361077912158
21.64361077912158
59.511642742048906
22.735360085145597
53.37045121830975
132.83289782977135
69.28361413222638
152.5745732607741
23.168645747733926
23.168645747733926
133.11385593265084
176.82272124751572
170.44673840704368
118.84400050761346
166.98829389674637
166.98829389674637
170.66747503945766
148.56625267380613
128.2626225804899
149.30892215850594
174.08705184437784
173.06382101581673
170.85445086330859
156.7889445158583
134.72712762622018
167.96396853418628
165.9230887627868
159.06017218051005
165.95138297411978
146.28036504762224
174.91054350793715
173.67901889032743
175.60929736159645
177.53372830648618
142.43135049736244
178.

In [7]:
df.head(200)

,label,leftElbow,rightElbow,leftShoulder,rightShoulder,leftKnee,rightKnee
0,goddess,105.850731,96.271785,78.997177,71.278112,104.706636,128.817581
1,goddess,105.134148,105.699485,103.970928,109.808665,119.696478,118.072152
2,goddess,179.519899,178.218694,22.352400,23.943679,108.062305,108.936671
3,goddess,145.021531,154.469908,137.264701,147.444288,112.854209,147.224011
4,goddess,15.159119,1.971805,55.746206,48.093558,81.315725,50.597415
...,...,...,...,...,...,...,...
195,warrior2,82.312149,83.986439,24.366191,35.392189,123.948933,134.124468
196,warrior2,177.262551,173.855873,80.879691,94.925114,178.315015,123.241560
197,warrior2,157.444196,146.403629,67.202168,71.066836,78.841928,176.191068
198,warrior2,150.871088,113.532492,14.898650,38.356366,177.457637,92.589901


In [8]:
from sklearn.svm import SVC
da = pd.read_csv("train_converted.csv")


In [9]:
X,Y= df.iloc[:,1:7],df['label']
model = SVC(kernel = 'poly')
model.fit(X,Y)

SVC(kernel='poly')

In [10]:
test= list()
test.append('label')
testdir = "/home/shikha/POSEESTIMATION/TEST"

In [11]:
test_images=0
test_data = pd.DataFrame(columns = cols)
for subdir, dirs, files in os.walk(testdir):
    for file in files:
        try:
            temp=[]
            one=[]
            imagename_test=os.path.basename(subdir)
            img_test = cv2.imread(testdir + "/" +imagename_test+"/" +file)
            imageWidth, imageHeight = img_test.shape[:2]
            imgRGB = cv2.cvtColor(img_test, cv2.COLOR_BGR2RGB)
            results = pose.process(imgRGB)
            if results.pose_landmarks:
                    mpDraw.draw_landmarks(img_test, results.pose_landmarks, mpPose.POSE_CONNECTIONS) #draw landmarks on image
                    landmarks = results.pose_landmarks.landmark
            
            one.append(imagename)
            leftElbow=calculate_angle([landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y],
                  [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y],
                   [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y])
            print(leftElbow)
            one.append(leftElbow)
            rightElbow=calculate_angle([landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y],
                       [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y],
                       [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y])
            one.append(rightElbow)


            leftShoulder=calculate_angle([landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y],
                       [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y],
                       [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y])
            one.append(leftShoulder)
            rightShoulder=calculate_angle([landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y],
                       [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y],
                       [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y])

            one.append(rightShoulder)
            leftKnee=calculate_angle([landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y],
                       [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y],
                       [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y])
            one.append(leftKnee)
            rightKnee=calculate_angle([landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y],
                       [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y],
                       [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y])
            one.append(rightKnee)
            if c==0:
                test_data=pd.DataFrame([list(one)],columns=cols)
                c=c+1
            else:
                test_data.loc[c]=one
                c=c+1
            test_images += 1
        
        except ValueError as ve:
            continue 
test_data.to_csv("test_converted.csv", index=False)

137.82226011331662
94.04401001592248
99.1136850972875
89.97196452473673
122.05763422308951
40.54755509024941
168.31960003220692
116.41029983882578
162.63517249359268
101.18362621613367
61.52870292286275
24.750464638736894
24.750464638736894
98.48647540293581
59.755060697556424
85.22112433511242
94.62449336500418
105.9305074705282
114.93235518842909
77.77361694913012
75.97377013165746
97.78909963365057
128.40532258357203
39.166899633579376
81.92938241528547
177.96329039680106
95.10650895655237
125.65282150096814
61.27791277818106
76.01284892135136
114.12490066244693
7.895430486556155
86.65475886670832
85.57060699233098
173.52905418538143
54.695924090504086
61.962683698740236
68.10382636979749
93.71454064623285
119.32031080581599
84.72199779706705
34.69259229667696
39.99327990654221
22.556562544540174
22.556562544540174
90.12937973779843
108.09976741055527
98.56830629262701
64.82837581431947
148.70496314402982
69.88365982904408
91.70600332180364
108.13132621998396
112.92100904363296
41.1

171.2067425405905
171.2067425405905
178.65183056182045
163.33553000691535
153.572812307597
148.82787948420423
133.41709235491382
176.94526393124158
40.815128568201175
175.9477736518337
129.25529463452358
169.7708926038653
167.83242390950173
88.31242374576397
172.59606337015387
177.7746430733107
157.2293759459625
174.4583310973629
174.4583310973629
174.13220285728912
174.13220285728912
161.74602384111944
169.46808774388828
170.06369168297311
153.91349795257662
154.92733878341824
175.72006000146104
156.119421284329
170.25073906699447
109.4014901482243
168.6024231526742
39.51952105970548
83.83952526180798
144.85403671221798
7.883093548703592
165.55491214807182
163.9671291119658
163.9671291119658
165.96846864111197
174.28565427983875
168.9092812982477
157.00454665464554
179.25790665968546
119.13154201607648
165.63146836486382
170.7837074887637
146.7648268506416
41.86292673311431
139.39826715004344
19.658857457859657
169.38689680097067
107.07727565150796
21.92076531739466
21.92076531739466


126.62402933406342
166.3220352679757
86.48964311556071
3.189126179926138
9.475364233854261
118.10037849780892
118.10037849780892
130.98736763365895
130.98736763365895
176.42296085469098
153.6923978572088
145.73786430443852
171.1427029290847
178.3404014251718
143.27678333664394
157.5063617468684
119.18539723693002
172.5296103156237
149.75849167180388
163.58623146167855
149.31053734099365
178.23526824621257
169.89255870086222
30.37061274088171
30.37061274088171
170.0802687080413
174.08089343064617
170.71551132461747
170.71551132461747
143.88565216043372
67.13189078367394
149.94785045250646
131.10241912319356
158.6220005424543
179.67556153551965
179.67556153551965
117.5576794908832
114.83322065750087
114.83322065750087
114.83322065750087
102.43030012422628
169.44356615853218
36.39075418308111
36.39075418308111
135.4724315060567
170.96813256649486
171.82683314838
171.82683314838
176.0069470307503
173.67756728595276
146.54679937264206
79.51806048260482
177.52977762628572


In [12]:

print(subdir)


# In[18]:


print(test_images)


/home/shikha/POSEESTIMATION/TEST/plank
944


In [13]:
test_data.head(20)

,label,leftElbow,rightElbow,leftShoulder,rightShoulder,leftKnee,rightKnee
1217,plank,137.822260,175.266571,43.313707,83.288641,168.565203,163.868566
1218,plank,94.044010,129.509723,29.650539,158.776961,153.607069,85.334494
1219,plank,99.113685,178.117457,107.409185,170.407373,157.767418,160.900289
1220,plank,89.971965,177.048149,88.164397,168.487875,30.185935,52.389074
1221,plank,122.057634,132.789790,54.294287,89.565442,3.450856,8.156114
1222,plank,40.547555,4.528116,20.073273,26.787008,146.727298,130.951377
1223,plank,168.319600,29.849702,11.750291,38.699568,175.107092,152.819341
1224,plank,116.410300,30.254172,30.694184,30.157474,86.102798,3.029451
1225,plank,162.635172,88.056906,19.931392,8.284880,166.378062,166.997041
1226,plank,101.183626,69.741345,72.479183,69.680233,139.086365,52.587272


In [15]:
S,T= test_data.iloc[:,1:7],test_data['label']
pred = model.predict(S)


In [16]:
print(pred)

['warrior2' 'warrior2' 'downdog' 'warrior2' 'downdog' 'goddess' 'No pose'
 'goddess' 'cobra' 'tree' 'goddess' 'goddess' 'goddess' 'goddess'
 'goddess' 'goddess' 'goddess' 'goddess' 'goddess' 'goddess' 'goddess'
 'goddess' 'No pose' 'goddess' 'goddess' 'cobra' 'goddess' 'goddess'
 'goddess' 'goddess' 'goddess' 'goddess' 'goddess' 'goddess' 'downdog'
 'goddess' 'goddess' 'goddess' 'goddess' 'goddess' 'goddess' 'goddess'
 'goddess' 'goddess' 'goddess' 'goddess' 'goddess' 'goddess' 'goddess'
 'downdog' 'goddess' 'goddess' 'downdog' 'goddess' 'goddess' 'goddess'
 'goddess' 'goddess' 'downdog' 'goddess' 'goddess' 'goddess' 'goddess'
 'goddess' 'goddess' 'cobra' 'downdog' 'goddess' 'downdog' 'goddess'
 'goddess' 'goddess' 'goddess' 'goddess' 'downdog' 'downdog' 'goddess'
 'goddess' 'goddess' 'No pose' 'downdog' 'downdog' 'goddess' 'goddess'
 'downdog' 'goddess' 'goddess' 'downdog' 'downdog' 'downdog' 'goddess'
 'downdog' 'goddess' 'warrior2' 'downdog' 'goddess' 'goddess' 'goddess'
 'goddess' 

In [17]:
from sklearn.metrics import accuracy_score
accuracy_score(T,pred)

0.03072033898305085